# ===============================================

In [ ]:
#already
!pip install SimpleITK
!pip install monai==1.4.0
!pip install einops==0.8.1
!pip install progress_table
!pip install medpy

In [ ]:
%cd /kaggle/input

# ===================================================================================================
### Testing

In [ ]:
import logging
import os
from datetime import datetime
import wandb
from tqdm import tqdm  # Import tqdm for progress bars
#Lib for preprocess + train
import SimpleITK as sitk
import torch
from sklearn.model_selection import KFold
from torch.utils.data.dataset import Dataset
import argparse
import random
import pathlib
import time
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import yaml
from monai.data import decollate_batch
import math
import shutil

import os
import cv2
from glob import glob
import numpy as np
import pandas as pd
import nibabel as nib
from matplotlib import colors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

#lib for model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch import optim
import math 
from torch.autograd import Variable
import einops
from einops import rearrange, repeat 
from einops.layers.torch import Rearrange, Reduce
from torch import nn, einsum
from monai.networks.blocks.dynunet_block import get_conv_layer
from monai.networks.blocks import UnetOutBlock
#for training
from functools import partial
from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference
from monai.transforms import (
    SpatialCrop,
    Activations,
    AsDiscrete,
    Compose,
    EnsureType
)

from monai.losses import DiceLoss #Minh co can code lai ham loss khong?
import monai.transforms as transforms
from monai.transforms import NormalizeIntensity

In [ ]:
#đáng lẽ hai cái này là hai cái khác nhau nhưng vì ở đây mình dùng bộ data đã có sẵn traningdata nên trùng

BRATS_TRAIN_FOLDERS = "/kaggle/input/brats2021/BraTS_2021_TrainingData/BraTS_2021_TrainingData"
DATA_PATH = "/kaggle/input/brats2021/BraTS_2021_TrainingData/BraTS_2021_TrainingData"
VALI_PATH = '/kaggle/input/brats2021/BraTS_2021_ValidationData/BraTS_2021_ValidationData'



DATA_TYPES = ['t1', 't1ce', 't2', 'flair', 'seg']
MASK_LABELS = ['Not Tumor', 'Non-Enhancing Tumor Core', 'Peritumoral Edema', 'GD-Enhancing Tumor']
MASK_VALUES = [0, 1, 2, 4]

In [ ]:
#preprocessing function
"""functions to correctly pad or crop non uniform sized MRI (before batching in the dataloader).
"""
def get_brats_folder(on="train"):
    if on == "train":
        return BRATS_TRAIN_FOLDERS
    else:
        return VALI_PATH
def pad_or_crop_image(image, seg=None, target_size=(128, 144, 144)):
    c, z, y, x = image.shape
    z_slice, y_slice, x_slice = [get_crop_slice(target, dim) for target, dim in zip(target_size, (z, y, x))]
    image = image[:, z_slice, y_slice, x_slice]
    if seg is not None:
        seg = seg[:, z_slice, y_slice, x_slice]
    todos = [get_left_right_idx_should_pad(size, dim) for size, dim in zip(target_size, [z, y, x])]
    padlist = [(0, 0)]  # channel dim
    for to_pad in todos:
        if to_pad[0]:
            padlist.append((to_pad[1], to_pad[2]))
        else:
            padlist.append((0, 0))
    image = np.pad(image, padlist)
    if seg is not None:
        seg = np.pad(seg, padlist)
        return image, seg
    return image

def get_left_right_idx_should_pad(target_size, dim):
    if dim >= target_size:
        return [False]
    elif dim < target_size:
        pad_extent = target_size - dim
        left = random.randint(0, pad_extent)
        right = pad_extent - left
        return True, left, right

def get_crop_slice(target_size, dim):
    if dim > target_size:
        crop_extent = dim - target_size
        left = random.randint(0, crop_extent)
        right = crop_extent - left
        return slice(left, dim - right)
    elif dim <= target_size:
        return slice(0, dim)
def get_crop_slice2(target_size, dim):
    if dim > target_size:
        start = (dim - target_size) // 2
        end = start + target_size
        return slice(start, end)
    elif dim <= target_size:
        return slice(0, dim)
def pad_or_crop_image_label(image, seg=None, target_size=(128, 128, 128)):
    c, z, y, x = image.shape
    z_slice, y_slice, x_slice = [get_crop_slice2(target, dim) for target, dim in zip(target_size, (z, y, x))]
    image = image[:, z_slice, y_slice, x_slice]
    if seg is not None:
        seg = seg[:, z_slice, y_slice, x_slice]
    
    # Pad nếu cần
    # todos = [get_left_right_idx_should_pad(size, dim) for size, dim in zip(target_size, [z, y, x])]
    # padlist = [(0, 0)]  # channel dim
    # for to_pad in todos:
    #     if to_pad[0]:
    #         padlist.append((to_pad[1], to_pad[2]))
    #     else:
    #         padlist.append((0, 0))
    
    # # Pad ảnh để đạt được target_size
    # image = np.pad(image, padlist)
    if seg is not None:
        # seg = np.pad(seg, padlist)
        return image, seg
    return image


def normalize(image):
    """Basic min max scaler.
    """
    min_ = np.min(image)
    max_ = np.max(image)
    scale = max_ - min_
    image = (image - min_) / scale
    return image

def irm_min_max_preprocess(image, low_perc=1, high_perc=99):
    """Main pre-processing function used for the challenge (seems to work the best).

    Remove outliers voxels first, then min-max scale.

    Warnings
    --------
    This will not do it channel wise!!
    """

    non_zeros = image > 0
    low, high = np.percentile(image[non_zeros], [low_perc, high_perc])
    image = np.clip(image, low, high)
    image = normalize(image)
    return image

def zscore_normalise(img):
    # Đảm bảo rằng img có kiểu dữ liệu float
    img = img.astype(np.float32)  # Nếu img là ndarray
    # img = img.to(torch.float32)  # Nếu img là tensor
    
    slices = slice(None)
    img[slices] = (img[slices] - np.mean(img[slices])) / np.std(img[slices])
    return img


In [ ]:
class Brats(Dataset):
    def __init__(self, patients_dir, benchmarking=False, training=True, data_aug=False,
                 no_seg=False, normalisation="zscore", normal=False):
        super(Brats, self).__init__()
        self.benchmarking = benchmarking
        self.normalisation = normalisation
        self.data_aug = data_aug
        self.training = training
        self.datas = []
        self.validation = no_seg
        self.normal = normal
        self.patterns = [".t1", ".t1ce", ".t2", ".flair"]
        self.target_spacing = [1.0, 1.0, 1.0]
        self.roi_size = [128, 128, 128]
        if self.training:
            self.transform = transforms.Compose([
                transforms.CropForegroundd(
                    keys=["image", "label"],
                    source_key="image",
                    k_divisible=self.roi_size,
                ),
                transforms.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0), #aug
                transforms.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1), #aug
                transforms.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2), #aug
                # transforms.NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
                transforms.RandScaleIntensityd(keys="image", factors=0.5, prob=1.0), #aug
                transforms.RandShiftIntensityd(keys="image", offsets=0.5, prob=1.0), #aug
            ])
        else:
            # Validation/Inference pipeline
            self.transform = transforms.Compose([
                transforms.CropForegroundd(
                    keys=["image", "label"],
                    source_key="image",
                    k_divisible=self.roi_size,
                )
            ])
        
        if not no_seg:
            self.patterns += [".seg"]
        for patient_dir in patients_dir:
            patient_id = patient_dir.name
            # Construct paths for each modality
            paths = [patient_dir / f"{patient_id}{value}.nii" for value in self.patterns]
            
            # Add patient data
            patient = dict(
                id=patient_id,
                t1=paths[0], t1ce=paths[1], t2=paths[2], flair=paths[3],
                seg=paths[4] if not no_seg else None
            )
            self.datas.append(patient)


    
        
    def __getitem__(self, idx):
        _patient = self.datas[idx]
        
        patient_image = {key: self.load_nii(_patient[key]) for key in _patient if key not in ["id", "seg"]} 
        #-> load t1 t1w t2 flair
        if _patient["seg"] is not None:
            patient_label = self.load_nii(_patient["seg"])
            et = patient_label == 4 # ET (Enhancing Tumor) - label 4
            et_present = 1 if np.sum(et) >= 1 else 0
            tc = np.logical_or(patient_label == 4, patient_label == 1) #TC (Tumor Core) - label 1, 4
            wt = np.logical_or(tc, patient_label == 2) # WT (Whole Tumor) label 1, 2, 4
            patient_label = np.stack([et, tc, wt])
        else:
            patient_label = np.zeros(patient_image.shape)  # placeholders, not gonna use it
            et_present = 0

        if self.normal == False: 
            if self.normalisation == "minmax":
                patient_image = {key: irm_min_max_preprocess(patient_image[key]) for key in patient_image}
            elif self.normalisation == "zscore":
                patient_image = {key: zscore_normalise(patient_image[key]) for key in patient_image}
        else:
            # patient_image = {key: zscore_normalise(patient_image[key]) for key in patient_image}
            normalize_intensity = NormalizeIntensity(nonzero=True, channel_wise=True)
            patient_image = {key: normalize_intensity(patient_image[key]) for key in patient_image}  # Chuẩn hóa cường độ cho mỗi ảnh

        patient_image = np.stack([patient_image[key] for key in patient_image])

        if self.training:
            z_indexes, y_indexes, x_indexes = np.nonzero(np.sum(patient_image, axis=0) != 0)
            zmin, ymin, xmin = [max(0, int(np.min(arr) - 1)) for arr in (z_indexes, y_indexes, x_indexes)]
            zmax, ymax, xmax = [int(np.max(arr) + 1) for arr in (z_indexes, y_indexes, x_indexes)]
            patient_image = patient_image[:, zmin:zmax, ymin:ymax, xmin:xmax]
            patient_label = patient_label[:, zmin:zmax, ymin:ymax, xmin:xmax]              
            patient_image, patient_label = pad_or_crop_image(patient_image, patient_label, target_size=(128, 128, 128))

            if self.normal == True:
                data_dict = {
                    "image": patient_image,
                    "label": patient_label,
                }
                transformed = self.transform(data_dict)
    
                patient_image = transformed["image"]
                patient_label = transformed["label"]
        else:
            z_indexes, y_indexes, x_indexes = np.nonzero(np.sum(patient_image, axis=0) != 0)
            zmin, ymin, xmin = [max(0, int(np.min(arr) - 1)) for arr in (z_indexes, y_indexes, x_indexes)]
            zmax, ymax, xmax = [int(np.max(arr) + 1) for arr in (z_indexes, y_indexes, x_indexes)]
            patient_image = patient_image[:, zmin:zmax, ymin:ymax, xmin:xmax]
            patient_label = patient_label[:, zmin:zmax, ymin:ymax, xmin:xmax]  
            patient_image, patient_label = pad_or_crop_image_label(patient_image, patient_label, target_size=(128, 128, 128))


        # Tiến hành các bước tiếp theo với transformed
        patient_image, patient_label = patient_image.astype("float16"), patient_label.astype("bool")
        patient_image, patient_label = [torch.from_numpy(x) for x in [patient_image, patient_label]]

        return dict(patient_id=_patient["id"],
                    image=patient_image, label=patient_label,
                    seg_path=str(_patient["seg"]) if self.training else str(_patient["t1"]),
                    crop_indexes=((zmin, zmax), (ymin, ymax), (xmin, xmax)),
                    et_present=et_present,
                    supervised=True,
                    idx=idx,
                    )

    @staticmethod
    def load_nii(path_folder):
        return sitk.GetArrayFromImage(sitk.ReadImage(str(path_folder)))

    def __len__(self):
        return len(self.datas)


In [ ]:
##### 
"""
Should check this

Get full data
"""


def get_datasets(seed, on="train", fold_number=0, normalisation="zscore", use_fold = False):
    base_folder_train = pathlib.Path(get_brats_folder(on)).resolve()
    base_folder_vali = pathlib.Path(get_brats_folder("a")).resolve()
    assert base_folder_train.exists()
    patients_dir_train = sorted([x for x in base_folder_train.iterdir() if x.is_dir()])
    patients_dir_vali = sorted([x for x in base_folder_vali.iterdir() if x.is_dir()])

    
    if use_fold == True:
        kfold = KFold(5, shuffle=True, random_state=seed)
    
        splits = list(kfold.split(patients_dir_train))
        train_idx, test_idx = splits[fold_number]
    
    
        train = [patients_dir_train[i] for i in train_idx]
        test = [patients_dir_train[i] for i in test_idx]
        train_dataset = Brats(train, training=True, normalisation=normalisation, normal = True)
        test_dataset = Brats(test, training=False, benchmarking=True, normalisation=normalisation, normal = True)
    else:
        train_dataset = Brats(patients_dir_train, training=True, normalisation=normalisation, normal = True)
        test_dataset = Brats(patients_dir_vali, training=False, benchmarking=True, normalisation=normalisation, normal = True)
        
    
    return train_dataset, test_dataset

full_train_dataset, val_dataset = get_datasets(123, fold_number=0)
print(len(full_train_dataset), len(val_dataset))

In [ ]:
"""
Create loader for neural network
"""


train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=2, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=2)

print("Train dataset number of batch:", len(train_loader))
print("Val dataset number of batch:", len(val_loader))

In [ ]:
"""
Test val_load and train_loader
"""

for batch in val_loader:
    # Assuming your input data is a 4D tensor (batch_size, channels, height, width)
    
    data_shape = batch['image'].shape
    label_shape = batch['label'].shape
    print("Data vali shape in the first batch:", data_shape)
    print("Label vali shape in the first batch:", label_shape)
    break  # Print only the first batch
for batch in train_loader:
    # Assuming your input data is a 4D tensor (batch_size, channels, height, width)
    
    data_shape = batch['image'].shape
    label_shape = batch['label'].shape
    print("Data train shape in the first batch:", data_shape)
    print("Label train shape in the first batch:", label_shape)
    break  # Print only the first batch

## Set up model

In [ ]:
class ProjectExciteLayer(nn.Module):
    """
        Project & Excite Module, specifically designed for 3D inputs
        *quote*
    """

    def __init__(self, num_channels, reduction_ratio=4):
        """
        :param num_channels: No of input channels
        :param reduction_ratio: By how much should the num_channels should be reduced
        """
        super(ProjectExciteLayer, self).__init__()
        num_channels_reduced = num_channels // reduction_ratio
        self.reduction_ratio = reduction_ratio
        self.relu = nn.GELU()
        self.conv_c = nn.Conv3d(in_channels=num_channels, out_channels=num_channels_reduced, kernel_size=1, stride=1)
        self.conv_cT = nn.Conv3d(in_channels=num_channels_reduced, out_channels=num_channels, kernel_size=1, stride=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        """
        :param input_tensor: X, shape = (batch_size, num_channels, D, H, W)
        :return: output tensor
        """
        batch_size, num_channels, D, H, W = input_tensor.size()

        # Project:
        # Average along channels and different axes
        squeeze_tensor_w = F.adaptive_avg_pool3d(input_tensor, (1, 1, W))

        squeeze_tensor_h = F.adaptive_avg_pool3d(input_tensor, (1, H, 1))

        squeeze_tensor_d = F.adaptive_avg_pool3d(input_tensor, (D, 1, 1))

        # tile tensors to original size and add:
        final_squeeze_tensor = sum([squeeze_tensor_w.view(batch_size, num_channels, 1, 1, W),
                                    squeeze_tensor_h.view(batch_size, num_channels, 1, H, 1),
                                    squeeze_tensor_d.view(batch_size, num_channels, D, 1, 1)])

        # Excitation:
        final_squeeze_tensor = self.sigmoid(self.conv_cT(self.relu(self.conv_c(final_squeeze_tensor))))
        output_tensor = torch.mul(input_tensor, final_squeeze_tensor)

        return output_tensor

In [ ]:
class DoubleConv(nn.Module):
    """(Conv3D -> BN -> ReLU) * 2"""
    def __init__(self, in_channels, out_channels, num_groups=16, kernel_size=3, padding=1, stride=1, bias=True):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, bias=bias),
            #nn.GroupNorm(num_groups=num_groups, num_channels=out_channels),
            nn.InstanceNorm3d(out_channels, affine = True),
            nn.LeakyReLU(),
            nn.Conv3d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, bias=bias),
            nn.InstanceNorm3d(out_channels, affine = True),
            #nn.GroupNorm(num_groups=num_groups, num_channels=out_channels),
            nn.LeakyReLU()
        )
    
    def forward(self, x):
        return self.double_conv(x)
class Downsampling(nn.Module):
    def __init__(self, in_channels, out_channels, 
        kernel_size, stride=1, padding=0, 
        norm=None):
        super().__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, 
                              stride=stride, padding=padding)
        
        self.norm = nn.InstanceNorm3d(out_channels, affine = True)#nn.GroupNorm(num_groups=8, num_channels=out_channels)

    def forward(self, x): #downsample(conv) --> post norm
        x = self.conv(x)
        x = self.norm(x)
        return x   

In [ ]:
def MBConv(
    dim_in,
    dim_out,
    *,
    expansion_rate = 2,
):
    hidden_dim = int(expansion_rate * dim_out)
    stride = 1

    net = nn.Sequential(
        nn.Conv3d(dim_in, hidden_dim, 1),
        #nn.GroupNorm(num_groups=8, num_channels=hidden_dim),
        nn.InstanceNorm3d(hidden_dim, affine = True),
        nn.GELU(),
        nn.Conv3d(hidden_dim, hidden_dim, 3, stride = stride, padding = 1, groups = hidden_dim),
        #nn.GroupNorm(num_groups=8, num_channels=hidden_dim),
        nn.InstanceNorm3d(hidden_dim, affine = True),
        nn.GELU(),
        ProjectExciteLayer(hidden_dim),
        nn.Conv3d(hidden_dim, dim_out, 1),
        #nn.GroupNorm(num_groups=8, num_channels=dim_out)
        nn.InstanceNorm3d(dim_out, affine = True),
    )
    return net

class GLUMBConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size=3,
        stride=1,
        expand_ratio=2
    ):
        super().__init__()

        mid_channels = round(in_channels * expand_ratio)

        self.act = nn.SiLU()
        self.norm = nn.InstanceNorm3d(out_channels, affine = True)#nn.GroupNorm(num_groups=8, num_channels=out_channels)
        self.Leaky = nn.LeakyReLU()
        self.inverted_conv = nn.Conv3d(in_channels,
                                       mid_channels,
                                       1,
                                       padding = 0,
                                       bias=True)
        self.depth_conv = nn.Conv3d(
            mid_channels,
            mid_channels,
            kernel_size,
            stride=stride,
            padding=1,
            groups=mid_channels,
            bias=True,
        )
        self.point_conv = nn.Conv3d(
            mid_channels//2,
            out_channels,
            1,
            padding = 0,
            bias=False
        )
        self.Gnorm = nn.InstanceNorm3d(mid_channels, affine = True)#nn.GroupNorm(num_groups=8, num_channels=mid_channels)
        self.PE = ProjectExciteLayer(num_channels = mid_channels//2)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.inverted_conv(x)
        x = self.Gnorm(x)
        x = self.Leaky(x)
        
        x = self.depth_conv(x)
        x = self.Gnorm(x)
        x = self.Leaky(x)
        x, gate = torch.chunk(x, 2, dim=1)
        gate = self.act(gate)
        x = x * gate
        x = self.PE(x)
        x = self.point_conv(x)
        x = self.norm(x)
        return x
class PreNormResidual(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x):
        return self.fn(self.norm(x)) + x

class FeedForward(nn.Module):
    def __init__(self, dim, mult = 4, dropout = 0.):
        super().__init__()
        inner_dim = int(dim * mult)
        self.net = nn.Sequential(
            nn.Linear(dim, inner_dim),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class PreNormResidual2(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.InstanceNorm3d(dim, affine = True)#nn.GroupNorm(num_groups=8, num_channels=dim)
        self.fn = fn

    def forward(self, x):
        return self.fn(self.norm(x)) + x

class FeedForward(nn.Module):
    def __init__(self, dim, mult = 3, dropout = 0.):
        super().__init__()
        inner_dim = int(dim * mult)
        self.net = nn.Sequential(
            nn.Linear(dim, inner_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
class Attention(nn.Module):
    def __init__(
        self,
        dim,
        dim_head = 32,
        dropout = 0.,
        window_size = (7,7,7)
    ):
        super().__init__()
        assert (dim % dim_head) == 0, 'dimension should be divisible by dimension per head'

        self.heads = dim // dim_head
        self.scale = dim_head ** -0.5
        self.to_qkv = nn.Linear(dim, dim * 3, bias = False)
        self.attend = nn.Sequential(
            nn.Softmax(dim = -1),
            nn.Dropout(dropout)
        )
        self.to_out = nn.Sequential(
            nn.Linear(dim, dim, bias = False),
            nn.Dropout(dropout)
        )
        # relative positional bias
        w1,w2,w3 = window_size
        # 初始化相对位置索引矩阵[2*H-1,2*W-1,2*D-1,num_heads]
        self.rel_pos_bias = nn.Embedding((2 * w1 - 1) *(2 * w2 - 1)*(2 * w3 - 1), self.heads)
        pos1 = torch.arange(w1)
        pos2 = torch.arange(w2)
        pos3 = torch.arange(w3)
        # 首先我们利用torch.arange和torch.meshgrid函数生成对应的坐标，[3,H,W,D] 然后堆叠起来，展开为一个二维向量，得到的是绝对位置索引。
        grid = torch.stack(torch.meshgrid(pos1, pos2, pos3, indexing = 'ij'))
        grid = rearrange(grid, 'c i j k -> (i j k) c')
        # 广播机制，分别在第一维，第二维，插入一个维度，进行广播相减，得到 3, whd*ww, whd*ww的张量
        rel_pos = rearrange(grid, 'i ... -> i 1 ...') - rearrange(grid, 'j ... -> 1 j ...') 
        rel_pos[...,0] += w1 - 1
        rel_pos[...,1] += w2 - 1
        rel_pos[...,2] += w3 - 1
        # 做了个乘法操作，以进行区分,最后一维上进行求和，展开成一个一维坐标   a*x1 + b*x2 + c*x3  (a= hd b=d c =1) 
        rel_pos_indices = (rel_pos * torch.tensor([(2 *w2 - 1)*(2 *w3 - 1), (2 *w3 - 1), 1])).sum(dim = -1)
        # 注册为一个不参与网络学习的变量
        self.register_buffer('rel_pos_indices', rel_pos_indices, persistent = False)
               

    def forward(self, x):
        batch, height, width, depth, window_height, window_width, window_depth ,_, device, h = *x.shape, x.device, self.heads
        # flatten
        x = rearrange(x, 'b x y z w1 w2 w3 d -> (b x y z) (w1 w2 w3) d')
        # project for queries, keys, values
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)
        # split heads
        q, k, v = map(lambda t: rearrange(t, 'b n (h d ) -> b h n d', h = h), (q, k, v))
        # scale
        q = q * self.scale
        # sim
        sim = einsum('b h i d, b h j d -> b h i j', q, k)
        # add positional bias
        bias = self.rel_pos_bias(self.rel_pos_indices)
        sim = sim + rearrange(bias, 'i j h -> h i j')
        # attention
        attn = self.attend(sim)
        # aggregate
        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        # merge heads
        out = rearrange(out, 'b h (w1 w2 w3) d -> b w1 w2 w3 (h d)', w1 = window_height, w2 = window_width, w3 = window_depth)
        # combine heads out
        out = self.to_out(out)
        return rearrange(out, '(b x y z) ... -> b x y z ...', x = height, y = width, z = depth)

In [ ]:
class MaxViT_Block(nn.Module):
    def __init__(
        self,
        *,
        dim = 512,
        dim_head = 32,
        window_size = (8,8,8),
        dropout = 0.1,
    ):
        super().__init__()
        w1,w2,w3 = window_size

        self.net = nn.Sequential(
            MBConv(dim, dim), # 1, 1, 1, 1, 8, 8, 8, 256
            Rearrange('b d (x w1) (y w2) (z w3) -> b x y z w1 w2 w3 d', w1 = w1, w2 = w2, w3 = w3),  # block-like attention -> [2, 1, 1, 1, 8, 8, 8, 256]
            PreNormResidual(dim, Attention(dim = dim, dim_head = dim_head, dropout = dropout, window_size = window_size)), #2, 1, 1, 1, 8, 8, 8, 256]
            PreNormResidual(dim, FeedForward(dim = dim, dropout = dropout)),
            Rearrange('b x y z w1 w2 w3 d -> b d (x w1) (y w2) (z w3)'),
            
            Rearrange('b d (x w1) (y w2) (z w3) -> b x y z w1 w2 w3 d', w1 = w1, w2 = w2, w3 = w3),  # block-like attention -> [2, 1, 1, 1, 8, 8, 8, 256]
            PreNormResidual(dim, Attention(dim = dim, dim_head = dim_head, dropout = dropout, window_size = window_size)), #2, 1, 1, 1, 8, 8, 8, 256]
            PreNormResidual(dim, FeedForward(dim = dim, dropout = dropout)),
            Rearrange('b x y z w1 w2 w3 d -> b d (x w1) (y w2) (z w3)'),
            )
    def forward(self, x):
        x = self.net(x)
        return x

In [ ]:
class Decoder1(nn.Module):
    def __init__(self, in_channels, out_channels, upsample_kernel_size, lka_size, kernel_size = 3, spatial_dims=3, test = True, i = 0):
        super().__init__()        

        upsample_stride = upsample_kernel_size
        self.upsample = nn.Upsample(scale_factor=upsample_kernel_size, mode='trilinear', align_corners=True)
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.Gnorm = nn.InstanceNorm3d(out_channels, affine = True)#nn.GroupNorm(num_groups=8, num_channels=out_channels)

        self.conv_block = nn.Conv3d(out_channels + out_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)

        self.point_wise_conv = nn.Sequential(
            nn.Conv3d(out_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.InstanceNorm3d(out_channels, affine = True),
            #nn.GroupNorm(num_groups=8, num_channels=out_channels)
        )

        self.depth_wise_conv = nn.Sequential(
            nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, groups=out_channels),
            nn.InstanceNorm3d(out_channels, affine = True),
            #nn.GroupNorm(num_groups=8, num_channels=out_channels),
            nn.LeakyReLU(negative_slope=0.01 )
        )
        self.conv_2 = nn.Conv3d(in_channels=1, out_channels=2, kernel_size=1)
        self.sigmoid = nn.Sigmoid() 
        self.Leaky = nn.LeakyReLU(negative_slope=0.01)
        self.PE = ProjectExciteLayer(num_channels = out_channels)
    def forward(self, x1, x2): #x2 = the skip connection
        skip = x2
        up1 = self.upsample(x1) #same as skip
        up1 = self.conv(up1)
        up1 = self.Gnorm(up1)
        x2 = self.depth_wise_conv(x2)
        x2 = torch.amax(x2, dim=1, keepdim=True) # 2, 1, 16, 16, 16   
        x1 = torch.amax(up1, dim=1, keepdim=True) # 2, 1, 16, 16, 16
        x1 = x1 + x2 #2 1 16 16 16 bang elementwise addition
        x1 = self.conv_2(x1) #-> 2 2 16 16 16
        x1 = F.softmax(x1, dim = 1) # 2 2 16 16 16
        x1,x2 = x1.split(1, dim = 1) #each = 2 1 16 16 16
        skip_channels = skip.size(1)
        x2 = x2.repeat(1,skip_channels,1,1,1)*skip # 2 128 16 16 16
        x2 = x2 + skip
        x1 = x1.repeat(1,skip_channels,1,1,1)*up1 + up1

        x1 = x1 * self.sigmoid(x2)
        x2 = x2 * self.sigmoid(x1)
       

        x1 = x1+x2 #co the thu cac phep toan khac o day
        x1 = self.point_wise_conv(x1) #out
        x1 = self.sigmoid(x1)
        x1 = skip * x1
        x1 = self.point_wise_conv(x1)#out
        x1 = self.PE(x1)
        x1 = self.sigmoid(x1)
        x1 = up1 * x1
        out = torch.cat((x1, skip), dim=1) #channel_x1 + channel_up1
        out = self.conv_block(out) #out_channel here 
        out = self.Leaky(out)
        return out

In [ ]:
# if __name__ == "__main__":
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # Create a random input tensor (Batch Size = 2, Channels = 256, H = 32, W = 32, D = 32)
#     img = torch.randn(2, 64, 32, 32, 32).to(device)
#     skip = torch.randn(2, 32, 64, 64, 64).to(device)

#     # Initialize the EfficientAttention block
#     efficient_attention = Up1(4*16, 2*16, upsample_kernel_size=2).to(device)#EfficientAttention(dim=32).to(device)
#     # efficient_attention = EfficientLinearAttention(dim=64).to(device).to(device)#EfficientAttention(dim=32).to(device)

#     # Run the forward pass to check if everything works
#     output = efficient_attention(img,skip)

#     # Print the output shape to verify
#     print(f"Output shape: {output.shape}")
#     total_params = sum(p.numel() for p in efficient_attention.parameters())
#     print(f"Total number of parameters: {total_params}")

In [ ]:
class Up2(nn.Module):
    def __init__(self, in_channels, out_channels, trilinear=False, test = True):
        super().__init__()
        spatial_dims = 3
        #self.upT = nn.ConvTranspose3d(in_channels, in_channels, kernel_size=2, stride=2)
        self.upsample = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)
        self.conv = nn.Conv3d(in_channels, in_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.convLK_in = nn.Sequential(
            nn.InstanceNorm3d(in_channels, affine = True),
            #nn.GroupNorm(num_groups=8, num_channels=in_channels),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv3d(in_channels, 2*out_channels, kernel_size=3, padding=1, stride=1, bias=False),
        )
        self.convLK_out = nn.Sequential(
            nn.InstanceNorm3d(2*out_channels, affine = True),#nn.GroupNorm(num_groups=8, num_channels=2*out_channels),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv3d(2*out_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False)
        )

        #self.channel_wise_maxpool = torch.amax(input_tensor, dim=1, keepdim=True)
    def forward(self, x1, x2): #x2 = the skip connection
        x1 = self.conv(self.upsample(x1))
        x1 = self.convLK_in(x1)#16 128 128 128
        x1 = self.convLK_out(x1)
        return x1

In [ ]:
#Downsampling + ProjectExciteLayer + LayerNormGeneral + LKA
class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels, lka_size):
        super().__init__()
        self.loop = lka_size
        self.encode1 = Downsampling(in_channels, out_channels, 
                                    kernel_size=3, stride=2, padding=1)
        self.MaxViT_Block = MaxViT_Block(dim = out_channels, #128                        # dimension of first layer, doubles every layer
                                         dim_head = 16,                    # dimension of attention heads, kept at 32 in paper
                                         window_size = (4,4,4),            # window size for block and grids 8 8 8 = out | 4 4 4 -> khong out
        )
    def forward(self, x):
        x = self.encode1(x)
        if self.loop == 21:
            x1 = self.MaxViT_Block(x)
        elif self.loop == 15:
            x1 = self.MaxViT_Block(x)
            # x1 = self.MaxViT_Block(x1)
        else:
            x1 = self.MaxViT_Block(x)
            x1 = self.MaxViT_Block(x1)
            # x1 = self.MaxViT_Block(x1)
        skip = x1
        return x1, skip


In [ ]:
class Bneck(nn.Module):
    def __init__(self, in_channels, out_channels, trilinear=False, test = True):
        super().__init__()
        self.encode1 = Downsampling(in_channels, out_channels,kernel_size=3, stride=2, padding=1)
        self.net = nn.Sequential(
            nn.Conv3d(out_channels, out_channels, 1),
            #nn.GroupNorm(num_groups=8, num_channels=out_channels),
            nn.InstanceNorm3d(out_channels, affine = True),
            nn.GELU(),
            nn.Conv3d(out_channels, out_channels, 3, stride = 1, padding = 1, groups = out_channels),
            nn.InstanceNorm3d(out_channels, affine = True),
            #nn.GroupNorm(num_groups=8, num_channels=out_channels),
            nn.GELU(),
        )

    def forward(self, x1): #x2 = the skip connection
        x1 = self.encode1(x1)
        x1 = self.net(x1)
        return x1

In [ ]:
class Unet(nn.Module):
    def __init__(self, in_channels, n_channels, n_classes): 
        super().__init__()
        self.in_channels = in_channels #4
        self.n_classes = n_classes 
        self.n_channels = n_channels


        self.conv = DoubleConv(in_channels, 2*n_channels, num_groups=8)
        self.enc1 = Encoder(2*n_channels, 4*n_channels, lka_size = 21) #64
        self.enc2 = Encoder(4*n_channels,  8*n_channels, lka_size = 15) #128
        self.enc3 = Encoder(8*n_channels, 16*n_channels, lka_size = 10) #256

        self.bottleneck = Bneck(16*n_channels, 32*n_channels) #512

        
        self.dec1 = Decoder1(32*n_channels, 16*n_channels, upsample_kernel_size=2, lka_size = 10) #concat(256|8 -> 128|16 (u1),skip) Out: 256|16 
        self.dec2 = Decoder1(16*n_channels, 8*n_channels, upsample_kernel_size=2, lka_size = 15)  #concat(128|16 -> 64|32 (u2),skip) Out: 128|32
        self.dec3 = Decoder1(8*n_channels, 4*n_channels, upsample_kernel_size=2, lka_size = 21)   #concat(64|32 -> 32|64 (u3),skip) Out: 64|64
        
        
        self.dec4 = Up2(4*n_channels, n_channels)    #32|64 -> 16|128 (u4)
        self.out = nn.Conv3d(in_channels=n_channels, out_channels=n_classes, kernel_size=1)
        self.apply(self.initialize_weights)
    def forward(self, x):
        x = self.conv(x)
        #---layer 1
        x_1,skip_conv1 = self.enc1(x) #Embedding
        #---layer 2
        x_1,skip_conv2 = self.enc2(x_1)
        #---layer 3 
        x_1, skip_conv3 = self.enc3(x_1) 
        
        #---bottleneck 
        x_1 = self.bottleneck(x_1)

        #---layer 3 
        x_out = self.dec1(x_1, skip_conv3) 
        #---layer 2 
        x_out = self.dec2(x_out, skip_conv2) 
        #---layer 1 
        x_out = self.dec3(x_out, skip_conv1) 
        #---layer output
        x_out = self.dec4(x_out,x) # 16|128 (u4)
        out = self.out(x_out) #3|128
        return out
    def initialize_weights(self, module):
            name = module.__class__.__name__.lower()
            if name in ["conv2d", "conv3d"]:
                nn.init.kaiming_normal_(module.weight)
            if hasattr(module, "bias") and module.bias is not None:
                nn.init.constant_(module.bias, 0)



model = Unet(in_channels=4, n_classes=3, n_channels=16).to('cuda')
print('Number of network parameters:', sum(param.numel() for param in model.parameters()))

## For the loss



In [ ]:

class EDiceLoss(nn.Module):
    """Dice loss tailored to Brats need.
    """

    def __init__(self, do_sigmoid=True):
        super(EDiceLoss, self).__init__()
        self.do_sigmoid = do_sigmoid
        self.labels = ["ET", "TC", "WT"]
        self.device = "cpu"

    def binary_dice(self, inputs, targets, label_index, metric_mode=False):
        smooth = 1e-5
        if self.do_sigmoid:
            inputs = torch.sigmoid(inputs)

        if metric_mode:
            inputs = inputs > 0.5
            if targets.sum() == 0:
                print(f"No {self.labels[label_index]} for this patient")
                if inputs.sum() == 0:
                    return torch.tensor(1., device="cuda")
                else:
                    return torch.tensor(0., device="cuda")
            # Threshold the pred
        intersection = EDiceLoss.compute_intersection(inputs, targets)
        if metric_mode:
            dice = (2 * intersection) / ((inputs.sum() + targets.sum()) * 1.0)
        else:
            dice = (2 * intersection + smooth) / (inputs.pow(2).sum() + targets.pow(2).sum() + smooth)
        if metric_mode:
            return dice
        return 1 - dice

    @staticmethod
    def compute_intersection(inputs, targets):
        intersection = torch.sum(inputs * targets)
        return intersection

    def forward(self, inputs, target):
        dice = 0
        ce = 0
        CE_L = torch.nn.BCELoss()
        for i in range(target.size(1)):
            dice = dice + self.binary_dice(inputs[:, i, ...], target[:, i, ...], i)
            ce = ce + CE_L(torch.sigmoid(inputs[:, i, ...]), target[:, i, ...])
        # dice_grad = torch.autograd.grad(dice, inputs, retain_graph=True)[0]
        # ce_grad = torch.autograd.grad(ce, inputs, retain_graph=True)[0]

        # # Tính toán tỷ lệ gradient
        # dice_grad_norm = dice_grad.norm(2)
        # ce_grad_norm = ce_grad.norm(2)

        # # Cập nhật trọng số dựa trên tỷ lệ gradient
        # gradient_ratio = dice_grad_norm / (ce_grad_norm + 1e-5)
        # self.weight_dice = min(1.0, max(0.0, self.weight_dice + 0.01 * gradient_ratio))
        # self.weight_ce = 1.0 - self.weight_dice
        
        # # Tính toán final loss với trọng số đã điều chỉnh
        # final_dice = (self.weight_dice * dice + self.weight_ce * ce) / target.size(1)
        # return final_dice
        final_dice = ( 0.75 * dice + 0.25 * ce) / target.size(1)
        return final_dice

    def metric(self, inputs, target):
        dices = []
        for j in range(target.size(0)):
            dice = []
            for i in range(target.size(1)):
                dice.append(self.binary_dice(inputs[j, i], target[j, i], i, True))
            dices.append(dice)
        return dices


class EDiceLoss_Val(nn.Module):
    """Dice loss tailored to Brats need.
    """

    def __init__(self, do_sigmoid=True):
        super(EDiceLoss_Val, self).__init__()
        self.do_sigmoid = do_sigmoid
        self.labels = ["ET", "TC", "WT"]
        self.device = "cpu"

    def binary_dice(self, inputs, targets, label_index, metric_mode=False):
        smooth = 1e-5
        if self.do_sigmoid:
            inputs = torch.sigmoid(inputs)

        if metric_mode:
            inputs = inputs > 0.5
            if targets.sum() == 0:
                print(f"No {self.labels[label_index]} for this patient")
                if inputs.sum() == 0:
                    return torch.tensor(1., device="cuda")
                else:
                    return torch.tensor(0., device="cuda")
            # Threshold the pred
        intersection = EDiceLoss_Val.compute_intersection(inputs, targets)
        if metric_mode:
            dice = (2 * intersection) / ((inputs.sum() + targets.sum()) * 1.0)
        else:
            dice = (2 * intersection + smooth) / (inputs.pow(2).sum() + targets.pow(2).sum() + smooth)
        if metric_mode:
            return dice
        return 1 - dice

    @staticmethod
    def compute_intersection(inputs, targets):
        intersection = torch.sum(inputs * targets)
        return intersection

    def forward(self, inputs, target):
        dice = 0
        for i in range(target.size(1)):
            dice = dice + self.binary_dice(inputs[:, i, ...], target[:, i, ...], i)
        final_dice = dice / target.size(1)
        return final_dice

    def metric(self, inputs, target):
        dices = []
        for j in range(target.size(0)):
            dice = []
            for i in range(target.size(1)):
                dice.append(self.binary_dice(inputs[j, i], target[j, i], i, True))
            dices.append(dice)
        return dices

class AverageMeter(object):
    """Computes and stores the average and current value."""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    @staticmethod
    def _get_batch_fmtstr(num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

### Traing

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
post_trans = Compose(
    [EnsureType(), Activations(sigmoid=True), AsDiscrete(argmax=False, threshold=0.5)]
)

post_sigmoid = Activations(sigmoid=True)
post_pred = AsDiscrete(argmax=False, threshold=0.5)
roi = (128, 128, 128) #128, 128, 128
sw_batch_size = 1
overlap = 0.5
VAL_AMP = True

wandb.login(key = "")

In [ ]:
def get_post_trans(tta=False):
    if tta:
        return Compose([
            EnsureType(),
            AsDiscrete(argmax=False, threshold=0.5)
        ])
    return Compose([
        EnsureType(),
        Activations(sigmoid=True),
        AsDiscrete(argmax=False, threshold=0.5)
    ])

#----------------
def train_epoch(model, loader, optimizer, epoch, loss_func, end, wandb_tracking = 0):
    model.train()
    run_loss = AverageMeter('Loss', ':.4e')

    num_steps = len(loader)
    print(f"Epoch {epoch}: Number of steps (batches) in this epoch: {num_steps}")

    for idx, batch_data in enumerate(loader):
        torch.cuda.empty_cache()
        data, target = batch_data["image"].float().cuda(), batch_data["label"].float().cuda()
        logits = model(data)

        loss = loss_func(logits, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        run_loss.update(loss.item(), n=batch_size)
        if wandb_tracking == 1:
            wandb.log({"train_step_loss": loss.item()})

    # Log the average loss for the epoch
    if wandb_tracking == 1:
        wandb.log({"train_epoch_loss": run_loss.avg, "epoch": epoch})
    return run_loss.avg

# ===============================================================================================
def model_inferer(input, model, tta=True, batch_size=2, overlap=0.6):
    def _compute(input, flip_dims=None):
        inputs = input if flip_dims is None else input.flip(dims=flip_dims)
        with torch.amp.autocast(device_type='cuda', enabled=VAL_AMP):
            output = sliding_window_inference(
                inputs=inputs,
                roi_size=roi,
                sw_batch_size=batch_size,
                predictor=model,
                overlap=overlap,
            )
        if flip_dims is not None:
            output = output.flip(dims=flip_dims)
        return output

    with torch.no_grad():
        if tta:
            # Danh sách các phép flip
            flip_combinations = [
                None,  # Gốc
                (2,),  # Flip x
                (3,),  # Flip y
                (4,),  # Flip z
                (2, 3),  # Flip x, y
                (2, 4),  # Flip x, z
                (3, 4),  # Flip y, z
                (2, 3, 4),  # Flip x, y, z
            ]
            predict = None
            for flip_dims in flip_combinations:
                output = _compute(input, flip_dims)
                output = torch.sigmoid(output)  # Chuyển logits thành xác suất
                predict = output if predict is None else predict + output
            predict = predict / 8.0  # Trung bình
        else:
            predict = _compute(input)  # Inference gốc, trả về logits

    torch.cuda.empty_cache()
    return predict


# ===============================================================================================

def evaluate_model(model, loader, epoch, acc_func, criterian_val, metric, wandb_tracking=0, tta=False):
    model.eval()
    run_acc = AverageMeter('Loss', ':.4e')
    all_preds = []
    all_labels = []
    post_trans = get_post_trans(tta)  # Chọn post_trans dựa trên tta
    with torch.no_grad():
        for idx, batch_data in enumerate(loader):
            val_inputs, val_labels = batch_data["image"].to(device), batch_data["label"].to(device)

            logits = model_inferer(val_inputs, model, tta=tta, batch_size=2, overlap=0.6)
            val_outputs_list = decollate_batch(logits)
            val_labels_list = decollate_batch(val_labels)

            # Áp dụng post_trans
            val_output_convert = [post_trans(val_pred_tensor).to(device) for val_pred_tensor in val_outputs_list]
            val_labels_convert = [label.to(device) for label in val_labels_list]
            del val_inputs, val_labels, logits, val_outputs_list, val_labels_list
            torch.cuda.empty_cache()
            all_preds.extend(val_output_convert)
            all_labels.extend(val_labels_convert)

            # Tính metrics
            acc_func.reset()
            acc_func(y_pred=val_output_convert, y=val_labels_convert)
            acc, not_nans = acc_func.aggregate()
            run_acc.update(acc.cpu().numpy(), n=not_nans.cpu().numpy())

        dice_et = run_acc.avg[0]
        dice_tc = run_acc.avg[1]
        dice_wt = run_acc.avg[2]

    # Log validation metrics
    if wandb_tracking == 1:
        wandb.log({
            "val_epoch_dice_et": dice_et,
            "val_epoch_dice_tc": dice_tc,
            "val_epoch_dice_wt": dice_wt,
            "epoch": epoch
        })
    return run_acc.avg

# ===============================================================================================


def trainer(model, train_loader, val_loader, optimizer, loss_func, 
            acc_func, criterian_val, metric, scheduler, tta=True, start_epoch=1, end_epoch=3, save_every=1, checkpoint_path=None, wandb_tracking = 0):
    # Initialize wandb logging
    if wandb_tracking:
        wandb.init(entity="uit-meow", project="medical", name="test-med1", config={
            "epochs": end_epoch,
            "optimizer": optimizer.__class__.__name__,
            "learning_rate": optimizer.param_groups[0]["lr"],
            "loss_func": loss_func.__class__.__name__,
            "scheduler": scheduler.__class__.__name__ if scheduler else None
        })
        wandb_table = wandb.Table(columns=["Epoch", "Train Loss", "Dice_ET", "Dice_TC", "Dice_WT", "val_avg_acc", "Best Model"])

    val_acc_max = 0.0
    best_epoch = 0
    TC_dices = []
    WT_dices = []
    ET_dices = []
    avg_dices = []
    loss_epochs, train_epochs = [], []

    # If checkpoint path is provided, load model and optimizer states
    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        if scheduler:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # Start from the next epoch
        val_acc_max = checkpoint['val_acc_max']  # Retain the best validation accuracy

        print(f"Resuming training from epoch {start_epoch}...")
    
    for epoch in range(start_epoch, end_epoch + 1):
        # Train the model for the current epoch
        train_loss = train_epoch(model, train_loader, optimizer, epoch=epoch, loss_func=loss_func, end=end_epoch, wandb_tracking = wandb_tracking)
        torch.cuda.empty_cache()

        # Update scheduler if available
        if scheduler is not None:
            scheduler.step()
        
        # Evaluate the model after every 'save_every' epoch or at the last epoch
        if epoch % save_every == 0 or epoch == end_epoch or epoch == start_epoch:
            loss_epochs.append(train_loss)
            train_epochs.append(epoch)
            val_acc = evaluate_model(model, val_loader, epoch=epoch,
                                    acc_func=acc_func, criterian_val=criterian_val, 
                                    metric=metric, wandb_tracking=wandb_tracking, tta=tta)
            ET_dice = val_acc[0]
            TC_dice = val_acc[1]
            WT_dice = val_acc[2]
            val_avg_acc = np.mean(val_acc)

            # Update dice coefficients
            ET_dices.append(ET_dice)
            TC_dices.append(TC_dice)
            WT_dices.append(WT_dice)
            avg_dices.append(val_avg_acc)

            # Check if this is the best model
            best_model_flag = False
            if val_avg_acc > val_acc_max:
                print(f"New best ({val_acc_max:.6f} --> {val_avg_acc:.6f}) at epoch {epoch}")
                val_acc_max = val_avg_acc
                best_epoch = epoch
                best_model_flag = True
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),  # Save scheduler state
                'val_acc_max': val_acc_max,
            }, save_current)  # Save model to 'model.pth'
            wandb_table.add_data(epoch, train_loss, ET_dice, TC_dice, WT_dice, val_avg_acc, "Yes" if best_model_flag else "No")
            if best_model_flag:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),  # Save scheduler state
                    'val_acc_max': val_acc_max,
                }, save_path)  # Save model to 'model.pth'
                

            torch.cuda.empty_cache()

                

    # ✅ Log table to wandb
    if wandb_tracking == 1:
        wandb.log({"Training Metrics": wandb_table})

    return (val_acc_max, TC_dices, WT_dices, ET_dices, avg_dices, loss_epochs, train_epochs)


In [ ]:
start = 0
end = 200#max_epochs
batch_size = 1
save_every = 2 #

learning_rate = 3e-4
weight_decay = 1e-5
checkpoint_path = "/kaggle/input/again21/pytorch/default/1/best_metric_model(1).pth"#/kaggle/input/scsenewsotalap/pytorch/default/1/scsenew_sotaLap.pth"
###
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "best_metric_model.pth")
save_current = os.path.join(save_dir, "current_model.pth")
###

criterion = EDiceLoss().cuda()
criterian_val = EDiceLoss_Val().cuda()
metric = criterian_val.metric

dice_acc = DiceMetric(include_background=True, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = end)


(val_acc_max, TC_dices, WT_dices, ET_dices, avg_dices, loss_epochs, train_epochs)  = trainer(model = model,
                                                                                            train_loader = train_loader,
                                                                                            val_loader = val_loader,
                                                                                            optimizer = optimizer,
                                                                                            loss_func = criterion,
                                                                                            acc_func = dice_acc,
                                                                                            criterian_val = criterian_val,
                                                                                            metric = metric,
                                                                                            scheduler = scheduler,
                                                                                            start_epoch = start,
                                                                                            end_epoch = end,
                                                                                            save_every = save_every,
                                                                                            checkpoint_path = checkpoint_path,
                                                                                            wandb_tracking = 1)

In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from monai.metrics import DiceMetric
from monai.metrics import HausdorffDistanceMetric
from medpy.metric import binary
from monai.data import decollate_batch
from monai.inferers import sliding_window_inference
from monai.transforms import (
    Activations,
    AsDiscrete,
    Compose,
    EnsureType
)

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
def val_epoch_hd95(model, loader, max_epochs, epoch, acc_func, metric, device):
    model.eval()
    run_acc = AverageMeter('Loss', ':.4e')
    run_acc1 = AverageMeter('Loss', ':.4e')
    hausdorff_metric = HausdorffDistanceMetric(include_background=True, reduction='mean_batch', get_not_nans=True)
    hd_metric = []
    hd95_metric = []
    with torch.no_grad():
        for idx, batch_data in enumerate(loader):
            val_inputs, val_labels = batch_data["image"].to(device), batch_data["label"].to(device)
            logits = model_inferer(val_inputs, model)

            val_outputs_list = decollate_batch(logits)
            val_labels_list = decollate_batch(val_labels)

            val_output_convert = [post_trans(val_pred_tensor) for val_pred_tensor in val_outputs_list]
            acc_func.reset()
            acc_func(y_pred=val_output_convert, y=val_labels_list)


            hausdorff_metric.reset()
            hausdorff_metric(y_pred=val_output_convert, y=val_labels_list)
            acc1, not_nans1 = hausdorff_metric.aggregate()
            run_acc1.update(acc1.cpu().numpy(), n=not_nans1.cpu().numpy())
            hausdorff_et = run_acc1.avg[0]
            hausdorff_tc = run_acc1.avg[1]
            hausdorff_wt = run_acc1.avg[2]

            segs = logits
            targets = val_labels
            hd = []
            hd95 = []
            dice = []
            for l in range(segs.shape[1]):
                if targets[0,l].cpu().numpy().sum() == 0:
                    hd.append(1)
                    hd95.append(0)
                    print((segs[0,l].cpu().numpy() > 0.5).sum())

                    continue
                if (segs[0,l].cpu().numpy() > 0.5).sum() == 0:
                    hd.append(0)
                    hd95.append(0)
                    continue

                hd.append(binary.hd(segs[0,l].cpu().numpy() > 0.5, targets[0,l].cpu().numpy() > 0.5, voxelspacing=None))

                hd95.append(binary.hd95(segs[0,l].cpu().numpy() > 0.5, targets[0,l].cpu().numpy() > 0.5, voxelspacing=None))
            hd_metric.append(hd)
            hd95_metric.append(hd95)



            hd_metric_mean = [np.nanmean(l) for l in zip(*hd_metric)]
            hd95_metric_mean = [np.nanmean(l) for l in zip(*hd95_metric)]
            acc, not_nans = acc_func.aggregate()
            run_acc.update(acc.cpu().numpy(), n=not_nans.cpu().numpy())
            dice_et = run_acc.avg[0]
            dice_tc = run_acc.avg[1]
            dice_wt = run_acc.avg[2]



    labels = ("ET", "TC", "WT")
    #metrics = {key: value for key, value in zip(labels, metrics)}

    hd_metric = [np.nanmean(l) for l in zip(*hd_metric)]
    hd95_metric = [np.nanmean(l) for l in zip(*hd95_metric)]
    print("hd_metric_final: ", hd_metric)
    print("hd95_metric_final: ", hd95_metric)

    return run_acc.avg

def calc_hd95(model, val_loader, device, max_epochs):
    dice_acc = DiceMetric(include_background=True, reduction='mean_batch', get_not_nans=True)
    criterian_val = EDiceLoss_Val().to(device)
    metric = criterian_val.metric

    val_acc = val_epoch_hd95(model, val_loader, max_epochs, epoch=0, acc_func=dice_acc, metric = metric, device=device)
    dice_et, dice_tc, dice_wt = val_acc[0], val_acc[1], val_acc[2]
    val_avg_acc = np.mean(val_acc)
    print(f"\t{'*' * 20}Epoch Summary{'*' * 20}")
    print(f"Final validation stats {1}/{max_epochs}, dice_et: {dice_et:.6f}, dice_tc: {dice_tc:.6f}, dice_wt: {dice_wt:.6f} , Dice_Avg: {val_avg_acc:.6f}")


calc_hd95(model, val_loader, device, end)

In [ ]:
epochs_data = [
    {
        "Epoch": train_epochs[i],
        "TC_dices": TC_dices[i] if i < len(TC_dices) else None,
        "WT_dices": WT_dices[i] if i < len(WT_dices) else None,
        "ET_dices": ET_dices[i] if i < len(ET_dices) else None,
        "avg_dices": avg_dices[i] if i < len(avg_dices) else None,
        "loss_epochs": loss_epochs[i] if i < len(loss_epochs) else None,
    }
    for i in range(len(train_epochs))
]

import pandas as pd

epochs_df = pd.DataFrame(epochs_data)
print(epochs_df)
